In [ ]:
import pandas as pd 
import geopandas
from shapely.geometry import Point
# df = pd.read_parquet("s3://alpha-everyone/platform_demos/performance/natstats_postcodes.parquet")
# dfs = df.sample(1000)
# dfs.to_parquet("postcodes.parquet")
postcodes = pd.read_parquet("postcodes.parquet")

In [ ]:
postcodes.head()

In [ ]:
postcodes["coords"] = list(zip(postcodes.LONG, postcodes.LAT))
postcodes['geometry'] = postcodes['coords'].apply(Point)

In [ ]:
geopostcodes = geopandas.GeoDataFrame(postcodes, geometry='geometry')

In [ ]:
uk_shape = geopandas.read_file("shapefile/GBR_adm0.shp")
uk_shape["geometry"].plot(figsize=(7,9))

In [ ]:
mywkt = uk_shape.loc[0, "geometry"].wkt


In [ ]:
# http://geoportal.statistics.gov.uk/datasets/local-authority-districts-december-2016-generalised-clipped-boundaries-in-the-uk?geometry=-83.187%2C45.365%2C135.134%2C63.013
import tempfile
from urllib.request import urlretrieve
import zipfile

tempdir = tempfile.TemporaryDirectory()
tempfile = tempfile.NamedTemporaryFile()

url = "https://opendata.arcgis.com/datasets/7ff28788e1e640de8150fb8f35703f6e_2.zip"
zipped_shp = urlretrieve(url, tempfile.name)

with zipfile.ZipFile(zipped_shp[0],"r") as zip_ref:
    zip_ref.extractall(tempdir.name)

shapefile_name = [f for f in os.listdir(tempdir.name) if f.endswith("shp")][0]
shapefile_path = os.path.join(tempdir.name, shapefile_name)

uk_lad_2016 = geopandas.read_file(shapefile_path)

# uk_lad_2016["geometry"].plot(figsize=(10,10))

In [ ]:
gpc = geopostcodes[["PCD", "geometry"]]
gpc.head()

In [ ]:
geopandas.sjoin(gpc, uk_lad_2016, op='within')